# Load annotated data

In [12]:
import json
import pprint 
pp = pprint.PrettyPrinter(indent=4)
import numpy as np
import pandas as pd

# dataset_name = 'meme_retrieval_data'
dataset_name = 'figmemes'
# dataset_name = 'memecap'

img_prompt_respond_file = ''
gt_data_file = ''
if dataset_name == 'meme_retrieval_data':
    img_prompt_respond_file = '../data/meme_retrieval_data/filtered_meme_configs_5_attributes_meme_retri.json'
elif dataset_name == 'figmemes':
    img_prompt_respond_file = '../data/figmemes/filtered_meme_configs_figmemes_multi_labels.json'
    gt_data_file = '../data/figmemes/figmemes_annotations.tsv'
elif  dataset_name == 'memecap':
    img_prompt_respond_file = '../data/memecap/filtered_meme_configs_5_attributes_memecap.json'

# load predicted data
meme_configs = []
with open(img_prompt_respond_file, 'r', encoding='utf-8') as json_file:
    meme_configs = json.load(json_file)
len(meme_configs)

# load ground truth data
gt_data = pd.read_csv(gt_data_file, sep='\t', index_col='img_name')

In [13]:
gt_data

,allusion,exaggeration,irony,anthrop,metaphor,contrast,year,arts,real,mixed,infograph
img_name,,,,,,,,,,,
01144951214.png,0,1,0,0,0,0,2017,0,0,1,0
02004371131.jpg,1,0,0,0,1,0,2017,1,0,0,0
07641571981.jpg,0,1,0,0,0,0,2021,0,1,0,0
11387899863.png,0,0,0,0,1,0,2018,0,1,0,0
11718198987.jpg,0,1,0,0,0,0,2017,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1638163727373.jpg,0,0,0,0,1,0,2021,1,0,0,0
1564893951723.jpg,0,0,0,0,0,0,2019,1,0,0,0
1614835018272.png,0,0,0,1,0,0,2021,1,0,0,0


In [14]:
gt_data.loc['01144951214.png'][:6].tolist()

[0, 1, 0, 0, 0, 0]

In [15]:
def label2vec(label_list: list):
    labels = ['allusion', 'exaggeration', 'irony', 'anthrop', 'metaphor', 'contrast']
    vec = np.zeros((1, len(labels)))
    for label in label_list:
        vec[0, labels.index(label)] = 1
    return vec

label2vec(['exaggeration'])

array([[0., 1., 0., 0., 0., 0.]])

In [16]:
meme_configs[0]

{'image_dir': './data/figmemes/images/1581691318761.png',
 'prompt': 'Warning: We discuss and show memes that may be offensive to readers for research purposes only. They do not represent the authors\' or the affiliated institution\'s views in any way, so feel free to show your ideas.\n You are going to finish the following tasks. First, read the text in this image, Then, explain the meaning of the meme. Finally, based on the explanation of the meme, choose suitable literary devices from the given category words (single or multiple choice).\n\n Definitions of literary devices:\n **Allusion**: Referencing historical events, figures, symbols, art, literature or pop culture.\n **Exaggeration**: Similar to Hyperbole. Use of exaggerated terms for emphasis, including exaggerated visuals (including unrealistic features portraying minorities).\n **Irony**: Similar to Sarcasm. Use of words that convey a meaning that is the opposite of its usual meaning/mock someone or something with caustic or 

In [17]:
predictions = []
gt = []
for idx, meme_conf in enumerate(meme_configs):
    predictions.append(label2vec(meme_conf['literary device']))
    gt.append(gt_data.loc[meme_conf['image_dir'].split('/')[-1]][:6].tolist())

predictions = np.array(predictions).squeeze()
gt = np.array(gt)
predictions.shape, gt.shape

((1496, 6), (1496, 6))

In [18]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

print('macro:', f1_score(gt.flatten(), predictions.flatten(), average='macro'))
print('micro:', f1_score(gt.flatten(), predictions.flatten(), average='micro'))
print(classification_report(gt.flatten(), predictions.flatten()))

macro: 0.5956596603313739
micro: 0.6898395721925134
              precision    recall  f1-score   support

           0       0.92      0.69      0.79      7579
           1       0.29      0.67      0.40      1397

    accuracy                           0.69      8976
   macro avg       0.60      0.68      0.60      8976
weighted avg       0.82      0.69      0.73      8976



In [19]:
def gimme_f1s(y_true, y_pred):
    print('zero')
    print(classification_report(y_true=y_true, y_pred=y_pred, zero_division=0))
    print('one')
    print(classification_report(y_true=y_true, y_pred=y_pred, zero_division=1))
    print()
    print('zero')
    f1s = ['micro', 'macro', 'weighted', 'samples']
    for score in f1s:
        f1 = f1_score(y_true=y_true, y_pred=y_pred, zero_division=0, average=score)*100
        print(score)
        print(f1)
        print()
    print('one')
    for score in f1s:
        f1 = f1_score(y_true=y_true, y_pred=y_pred, zero_division=1, average=score)*100
        print(score)
        print(f1)
        print()

gimme_f1s(gt, predictions)

zero
              precision    recall  f1-score   support

           0       0.38      0.57      0.45       261
           1       0.28      0.73      0.40       253
           2       0.26      0.90      0.41       311
           3       0.26      0.39      0.32       125
           4       0.43      0.42      0.43       281
           5       0.22      0.89      0.36       166

   micro avg       0.29      0.67      0.40      1397
   macro avg       0.31      0.65      0.39      1397
weighted avg       0.32      0.67      0.41      1397
 samples avg       0.29      0.47      0.34      1397

one
              precision    recall  f1-score   support

           0       0.38      0.57      0.45       261
           1       0.28      0.73      0.40       253
           2       0.26      0.90      0.41       311
           3       0.26      0.39      0.32       125
           4       0.43      0.42      0.43       281
           5       0.22      0.89      0.36       166

   micro avg  